<a href="https://colab.research.google.com/github/zhuxingyu0096/model/blob/main/multi_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from transformers import CLIPProcessor, CLIPModel
import torch
import os



In [2]:
#FuseCap
# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 加载处理器和模型
processor1 = BlipProcessor.from_pretrained("noamrot/FuseCap")
model1 = BlipForConditionalGeneration.from_pretrained("noamrot/FuseCap").to(device)


#Blip
pipe1 = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")
# 加载模型和处理器
processor2 = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model2 = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

#Vit-gpt2
# 加载模型
model3 = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model3.to(device)


#Clip
# 加载CLIP模型和处理器
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

In [3]:
# 指定当前目录下的图片路径
image_path = '/content/cat.JPEG'  # 替换为你的图片文件名

if not os.path.exists(image_path):
    raise FileNotFoundError(f"Image file {image_path} not found.")

# 加载图片
raw_image = Image.open(image_path).convert('RGB')

In [26]:
#FuseCap
#无条件图像字幕生成
inputs = processor1(raw_image, return_tensors="pt").to(device)

out = model1.generate(**inputs, num_beams=3)
generated_text1 = processor1.decode(out[0], skip_special_tokens=True).strip()

# 去掉开头的逗号
if generated_text1.startswith(','):
    generated_text1 = generated_text1[1:].strip()

print(generated_text1)


#Blip
#无条件图像字幕生成
inputs = processor2(raw_image, return_tensors="pt")

out = model2.generate(**inputs, max_new_tokens=50)
generated_text2 = processor2.decode(out[0], skip_special_tokens=True).strip()
print(generated_text2)


#Vit-gpt2
max_length = 30
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

def predict_step(image_path):
    # 加载单张图片
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file {image_path} not found.")

    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
        i_image = i_image.convert(mode="RGB")

    # 处理图片并生成描述
    pixel_values = feature_extractor(images=[i_image], return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model3.generate(pixel_values, **gen_kwargs)
    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

    # 去掉前后的符号
    if preds:
        return preds[0].strip("[]")  # 取第一个元素并去掉前后的方括号
    return None

# 替换为你当前目录下的实际图片文件名
generated_text3 = predict_step('/content/cat.JPEG')
print(generated_text3)



a cat with green eyes and pointy ears looks out of a window framed by brown curtains, with a white leg visible in the foreground
araffe cat sitting on a bed looking out a window
a cat sitting on top of a bed 


In [34]:
#CLIP取最优描述
# 加载图像
image = Image.open(image_path)
# 组合描述列表
descriptions = [generated_text1, generated_text2, generated_text3]

# 处理图像和描述
inputs = processor(text=descriptions, images=image, return_tensors="pt", padding=True)

# 获取图像和文本的特征向量
with torch.no_grad():
    image_features = model.get_image_features(inputs['pixel_values'])
    text_features = model.get_text_features(inputs['input_ids'])

# 计算余弦相似度
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (image_features @ text_features.T).squeeze(0)

# 找到最接近的描述
closest_description_index = similarity.argmax().item()
closest_description = descriptions[closest_description_index]

print("与图片最接近的描述是:", closest_description)



与图片最接近的描述是: a cat with green eyes and pointy ears looks out of a window framed by brown curtains, with a white leg visible in the foreground
